<h1>A Toy Wave Equation</h1>
Here we build a very small Cactus application to evolve the wave equation. The main new thing we are learning
about here is the MoL (Method of Lines) thorn. This thorn abstracts the time integration of your simulation
from the physics code.

In [ ]:
%cd ~/CactusFW2

We're going through the boilerplate of making a new thorn, but we'll do some special
things along the way.

In [ ]:
# Define some basic parameters describing a new thorn
thorn_pars = {
   "thorn_name" : "WaveToyET18",
   "arrangement_name" : "FunwaveUtils",
   "author" : "Steven R. Brandt",
   "email" : "sbrandt@cct.lsu.edu",
   "license" : "BSD"
 }
import os
os.environ["ARR"]=thorn_pars["arrangement_name"]
os.environ["THORN"]=thorn_pars["thorn_name"]

This next command is only needed if you want to delete the thorn you are
going to create below and start over.

In [ ]:
!rm -fr arrangements/$ARR/$THORN

Next we define some functions to help us create files automatically.

In [ ]:
import re
# This function does substitutes all occurances of "{name}" in input_str with values["name"]
# and returns the new string.
def replace_values(input_str,values):
    while True:
        g = re.search(r'{(\w+)}',input_str)
        if g:
            var = g.group(1)
            if var in values:
                val = values[var]
            else:
                raise Exception("Undefined: <<"+var+">>")
            start = g.start(0)
            end = start+len(g.group(0))
            input_str = input_str[0:start]+val+input_str[end:]
            continue
        break
    return input_str

# Createas a file with the given name, uses replace_values()
# to update file_contents with file_values, and writes it out.
def create_file(file_name,file_contents,file_values):
    fd = open(file_name,"w")
    file_contents = replace_values(file_contents,file_values)
    print("Over-writing file '"+file_name+"'")
    file_contents = re.sub(r'^\s+','',file_contents)
    fd.write(file_contents)
    fd.close()
    
# The equivalent of mkdir -p
def create_dir(dir):
    print("Ensuring directory '"+dir+"'")
    if not os.path.exists(dir):
        os.makedirs(dir)

<h3>Something new!</h3>
In the interface.ccl file we not only declare a grid function, but we give it scratch space of three time levels.
This gives MoL something to work with. Different time integrators will need different numbers of time levels.

Perhaps also unexpected are the rhs variables. The phi_rhs will hold the time derivative of phi, the psi_rhs will
hold the time derivative of psi. MoL will call the evolution routine repeatedly, storing the intermediate steps as
necessary for its time integration.

But wait! There's more! We also declare that we require a method called "MoLRegisterEvolvedGroup"
be provided by some thorn.

In [ ]:
interface_ccl_contents = """
## Interface definitions for thorn {thorn_name}
inherits: grid
## An implementation name is required for all thorns. No
## two thorns in a configuration can implement the same
## interface.
implements: {thorn_name}

## the groups declared below can be public, private, or protected.
public:

## A group defines a set of variables that are allocated together
## and share common properties, i.e. timelevels, tags such as the
## Prolongation=None tag. The type tag can take on the values
## GF, Scalar, or Array.

## Note that the number of timelevels can be an integer parameter
## GF stands for "Grid Function" and refers to a distributed array
## data structure.
#cctk_real force_group type=GF timelevels=3 tags='Prolongation="None"'
#{
#  force1, force2
#}

## Scalars are single variables that are available on all processors.
#cctk_real scalar_group type=SCALAR 
#{
#  scalar1, scalar2
#}

cctk_real evol_group type=GF timelevels=3
{
  psi, phi
}
cctk_real rhs_group type=GF 
{
  psi_rhs, phi_rhs
}

#----------------------------------------------------
#   MoL functions to register variables
#----------------------------------------------------

CCTK_INT FUNCTION MoLRegisterEvolvedGroup(CCTK_INT IN EvolvedIndex, CCTK_INT IN RHSIndex)
REQUIRES FUNCTION MoLRegisterEvolvedGroup

"""

<h3>Something new!</h3>
We schedule three routines. The initial and evolution routines are unsurprising. The register method, however, is
new. This is the place where we will tell MoL what variables we are evolving.

In [ ]:
schedule_ccl_contents = """
## Schedule definitions for thorn {thorn_name}

## There won't be any storage allocated for a group
## unless a corresponding storage declaration exists
## for it in the schedule file. In square brackets,
## we specify the number of storage levels to allocate.
## These commented examples correspond to the commented
## examples in the interface file above.
# storage: force_group[3]
# storage: scalar_group

## Schedule a function defined in this thorn to run at the beginning
## of the simulation. The minimum you need to specify for a schedule
## item is what language it's written in. Choices are: C (which includes
## C++) and Fortran (which means Fortran90).
#SCHEDULE init_function at CCTK_INIT
#{
#  LANG: C
#}"Do some initial stuff"

storage: evol_group[3], rhs_group

SCHEDULE init_wave_toy in CCTK_INITIAL
{
  LANG: C
} "initialize data"

SCHEDULE evolve_wave_toy in MoL_CalcRHS
{
  LANG: C
} "evolve data"

schedule wave_toy_registervars in MoL_Register
{
  LANG: C
}"Register funwave variables for MoL"

"""

In [ ]:
param_ccl_contents = """
## Parameter definitions for thorn {thorn_name}
## There are five types of parameters: int, real, keyword, string, and boolean.
## The comments provide prototypes of each.
#
#CCTK_INT one_to_five "This integer parameter goes from 1 to 5"
#{
#  1:5 :: "Another comment"
#} 3 # This is the default value
#
#CCTK_REAL from_2p5_to_3p8e4 "This integer parameter goes from 2.5 to 3.8e4"
#{
#  2.5:3.8e4 :: "Another comment"
#} 4.4e3 # This is the default value
#
## This keyword example defines the parameter wavemaker_type and 8 possible values.
#CCTK_KEYWORD wavemaker_type "types of wave makers"
#{
#  "ini_rec" :: "initial rectangular hump, need xc,yc and wid"
#  "lef_sol" :: "initial solitary wave, WKN B solution, need amp, dep"
#  "ini_oth" :: "other initial distribution specified by users"
#  "wk_reg" :: "Wei and Kirby 1999 internal wave maker, need xc_wk, tperiod, amp_wk, dep_wk, theta_wk, and time_ramp (factor of period)"
#  "wk_irr" :: "Wei and Kirby 1999 TMA spectrum wavemaker, need xc_wk, dep_wk, time_ramp, delta_wk, freqpeak, freqmin, freqmax, hmo, gammatma, theta-peak"
#  "wk_time_series" :: "fft a time series to get each wave component and then use Wei and Kirby's ( 1999) wavemaker. Need input wavecompfile (including 3 columns:   per,amp,pha) and numwavecomp, peakperiod, dep_wk, xc_wk, ywidth_wk"
#  "ini_gau" :: "initial Gaussian hump, need amp, xc, yc, and wid"
#  "ini_sol" :: "initial solitary wave, xwavemaker"
#}"wk_reg" # This is the default value
#
#CCTK_STRING a_string_par "a comment"
#{
#  .* :: "This is a perl 5 regular expression defining what the string may contain"
#} "blah blah blah" # This is the default value
#
#BOOLEAN a_boolean_par "a comment"
#{
#} true


"""

In [ ]:
configuration_ccl_contents = """
# Configuration definitions for thorn {thorn_name}
## You should not need include "mpi.h", but if you
## do, you will need this next line.
# REQUIRES MPI
# REQUIRES HDF5
"""

In [ ]:
makefile_contents = """
# Main make.code.defn file for thorn {thorn_name}

# Source files in this directory
SRCS = init.cc evolve.cc molregister.cc

# Subdirectories containing source files
SUBDIRS =
"""

In [ ]:
readme_contents = """
Author(s)    : {author} <{email}>
Maintainer(s): {author} <{email}>
Licence      : {license}
--------------------------------------------------------------------------

1. Purpose

not documented
"""

In [ ]:
import os

# This function will create a complete thorn
def create_thorn():
  # Create the thorn directory inside the Cactus source tree
  arrangement_dir = "arrangements/"+thorn_pars["arrangement_name"]
  thorn_dir = arrangement_dir + "/" + thorn_pars["thorn_name"]
  create_dir(thorn_dir)

  # Create basic ccl files needed by Cactus
  schedule_ccl = thorn_dir+"/schedule.ccl"
  interface_ccl = thorn_dir+"/interface.ccl"
  param_ccl = thorn_dir+"/param.ccl"
  configuration_ccl = thorn_dir+"/configuration.ccl"
  create_file(schedule_ccl,schedule_ccl_contents,thorn_pars)
  create_file(interface_ccl,interface_ccl_contents,thorn_pars)
  create_file(param_ccl,param_ccl_contents,thorn_pars)
  create_file(configuration_ccl,configuration_ccl_contents,thorn_pars)

  # Create the source directory and first makefile
  src_dir = thorn_dir+"/src"
  create_dir(src_dir)
  create_file(src_dir+"/make.code.defn",makefile_contents,thorn_pars)

  # Other dirs and files not strictly needed
  # for compiling and running Cactus.
  create_file(thorn_dir+"/README",readme_contents,thorn_pars)
  
  test_dir = thorn_dir+"/test"
  create_dir(test_dir)
  par_dir = thorn_dir+"/par"
  create_dir(par_dir)
  doc_dir = thorn_dir+"/doc"
  create_dir(doc_dir)

In [ ]:
create_thorn()

In [ ]:
my_thorns_contents="""
# ./configs/sim/ThornList
# This file was automatically generated using the GetComponents script.

!CRL_VERSION = 2.0


# Component list: funwave.th

!DEFINE ROOT = CactusFW2
!DEFINE ARR = $ROOT/arrangements
!DEFINE ET_RELEASE = trunk
!DEFINE FW_RELEASE = FW_2014_05

#Cactus Flesh
!TARGET   = $ROOT
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactus.git
!NAME     = flesh
!CHECKOUT = CONTRIBUTORS COPYRIGHT doc lib Makefile src

!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/stevenrbrandt/cajunwave.git
!REPO_PATH= $2
# Old version
#!AUTH_URL = https://svn.cct.lsu.edu/repos/projects/ngchc/code/branches/$FW_RELEASE/$1/$2
#!URL = https://svn.cct.lsu.edu/repos/projects/ngchc/code/branches/$FW_RELEASE/$2
#!URL = https://svn.cct.lsu.edu/repos/projects/ngchc/code/CactusCoastal/$2
!CHECKOUT =
CactusCoastal/Funwave
CactusCoastal/FunwaveMesh
CactusCoastal/FunwaveCoord
CactusCoastal/Tridiagonal
CactusCoastal/Tridiagonal2

# CactusBase thorns
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactusbase.git
!REPO_PATH= $2
!CHECKOUT =
CactusBase/Boundary
CactusBase/CartGrid3D
CactusBase/CoordBase
CactusBase/Fortran
CactusBase/InitBase
CactusBase/IOASCII
CactusBase/IOBasic
CactusBase/IOUtil
CactusBase/SymBase
CactusBase/Time
#
# CactusNumerical thorns
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactusnumerical.git
!REPO_PATH= $2
!CHECKOUT =
!CHECKOUT =
CactusNumerical/MoL
CactusNumerical/LocalInterp

CactusNumerical/Dissipation
CactusNumerical/SpaceMask
CactusNumerical/SphericalSurface
CactusNumerical/LocalReduce
CactusNumerical/InterpToArray

!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactusutils.git
!REPO_PATH= $2
!CHECKOUT = CactusUtils/Accelerator CactusUtils/OpenCLRunTime
CactusUtils/NaNChecker
CactusUtils/Vectors
CactusUtils/SystemTopology

# Carpet, the AMR driver
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/eschnett/carpet.git
!REPO_PATH= $2
!CHECKOUT = Carpet/doc
Carpet/Carpet
Carpet/CarpetEvolutionMask
Carpet/CarpetIOASCII
Carpet/CarpetIOBasic
Carpet/CarpetIOHDF5
Carpet/CarpetIOScalar
#Carpet/CarpetIntegrateTest
Carpet/CarpetInterp
Carpet/CarpetInterp2
Carpet/CarpetLib
Carpet/CarpetMask
#Carpet/CarpetProlongateTest
Carpet/CarpetReduce
Carpet/CarpetRegrid
Carpet/CarpetRegrid2
#Carpet/CarpetRegridTest
Carpet/CarpetSlab
Carpet/CarpetTracker
Carpet/CycleClock
#Carpet/HighOrderWaveTest
Carpet/LoopControl
#Carpet/ReductionTest
#Carpet/ReductionTest2
#Carpet/ReductionTest3
#Carpet/RegridSyncTest
Carpet/TestCarpetGridInfo
Carpet/TestLoopControl
Carpet/Timers

# Additional Cactus thorns
!TARGET   = $ARR
!TYPE     = svn
!URL      = https://svn.cactuscode.org/projects/$1/$2/trunk
!CHECKOUT = ExternalLibraries/OpenBLAS ExternalLibraries/OpenCL ExternalLibraries/pciutils ExternalLibraries/PETSc
ExternalLibraries/MPI
ExternalLibraries/HDF5
ExternalLibraries/zlib
ExternalLibraries/hwloc

# Simulation Factory
!TARGET   = $ROOT/simfactory
!TYPE     = git
!URL      = https://bitbucket.org/simfactory/simfactory2.git
!NAME     = simfactory2
!CHECKOUT = README.md README_FIRST.txt bin doc etc lib mdb

# Various thorns from LSU
#!TARGET   = $ARR
#!TYPE     = git
#!URL      = https://bitbucket.org/einsteintoolkit/archivedthorns-vectors.git
#!REPO_PATH= $2
#!CHECKOUT =
#LSUThorns/Vectors
#LSUThorns/QuasiLocalMeasures
#LSUThorns/SummationByParts
#LSUThorns/Prolong

#Roland/MapPoints
#Tutorial/BadWaveMoL
#Tutorial/BasicWave
#Tutorial/BasicWave2
#Tutorial/BasicWave3

# Various thorns from the AEI
# Numerical
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/numerical.git
!REPO_PATH= $2
!CHECKOUT =
#AEIThorns/ADMMass
AEIThorns/AEILocalInterp
#AEIThorns/PunctureTracker
#AEIThorns/SystemStatistics
#AEIThorns/Trigger
{arrangement_name}/{thorn_name}"""
create_file("my_thorns.th",my_thorns_contents,thorn_pars)

<h3>Something new!</h3>
Here we create the source code. The initialization, evolution, and molregister functions.

In [ ]:
src_dir = "arrangements/"+thorn_pars["arrangement_name"]+"/"+thorn_pars["thorn_name"]+"/src"

init_cc_src = """
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>
#include <math.h>

void init_wave_toy(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  for(int k=0;k<cctk_lsh[2];k++) {
      for(int j=0;j<cctk_lsh[1];j++) {
          for(int i=0;i<cctk_lsh[0];i++) {
              int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
              CCTK_REAL xx = x[cc] - 5.0;
              CCTK_REAL yy = y[cc] - 5.0;
              psi[cc] = exp(-xx*xx-yy*yy);
              phi[cc] = 0;
          }
      }
  }
}
"""
create_file(src_dir+"/init.cc",init_cc_src,{})

In [ ]:
evolve_cc_src = """
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>
#include <math.h>
#include <assert.h>

void evolve_wave_toy(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  for(int k=0;k<cctk_lsh[2];k++) {
      for(int j=1;j<cctk_lsh[1]-1;j++) {
          for(int i=1;i<cctk_lsh[0]-1;i++) {
              int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
              int im1 = CCTK_GFINDEX3D(cctkGH,i-1,j,k);
              int ip1 = CCTK_GFINDEX3D(cctkGH,i+1,j,k);
              int jm1 = CCTK_GFINDEX3D(cctkGH,i,j-1,k);
              int jp1 = CCTK_GFINDEX3D(cctkGH,i,j+1,k);
              CCTK_REAL dx = CCTK_DELTA_SPACE(0);
              CCTK_REAL dy = CCTK_DELTA_SPACE(1);
              psi_rhs[cc] = phi[cc];
              CCTK_REAL divx = (psi[ip1] + psi[im1] - 2*psi[cc])/(dx*dx);
              CCTK_REAL divy = (psi[jp1] + psi[jm1] - 2*psi[cc])/(dy*dy);
              phi_rhs[cc] = divx + divy;
          }
      }
  }
}
"""
create_file(src_dir+"/evolve.cc",evolve_cc_src,{})

In [ ]:
molregister_cc_src = """
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>

extern "C" void wave_toy_registervars (CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  CCTK_INT ierr = 0, group, rhs;

  // Look up the group index of the evol_group
  group = CCTK_GroupIndex ("{thorn_name}::evol_group");
  
  // Look up the group index of the rhs_group
  rhs = CCTK_GroupIndex ("{thorn_name}::rhs_group");

  if (CCTK_IsFunctionAliased ("MoLRegisterEvolvedGroup"))
  {
    // *** KEY STEP! *** Tell MoL what we variables
    // we are evolving and what the rhs variables are.
    ierr += MoLRegisterEvolvedGroup (group, rhs);
  }
  else
  {
    CCTK_WARN (0, "MoL function not aliased !");
    ierr++;
  }

  if (ierr)
    CCTK_WARN (0, "Problems registering with MoL !");
}

"""
create_file(src_dir+"/molregister.cc",molregister_cc_src,thorn_pars)

In [ ]:
!time ./simfactory/bin/sim build --mdbkey make 'make -j2' --thornlist=./my_thorns.th | cat -

In [ ]:
# A minimal par file....

wave_toy_par_src = """
ActiveThorns = "{thorn_name} coordbase carpet cartgrid3d MoL"

Cactus::cctk_itlast = 250

# the output dir will be named after the parameter file name
IO::out_dir = $parfile

# The set of parameters required to specify the grid...
Carpet::domain_from_coordbase = "yes"
CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin =  0
CoordBase::xmax =  30
CoordBase::ymin =  0
CoordBase::ymax =  30
CoordBase::zmin =  0.0
CoordBase::zmax =  0.0
CoordBase::dx   =  0.25
CoordBase::dy   =  0.25

# Because we're doing a 2D code...
CoordBase::boundary_size_z_lower     = 0
CoordBase::boundary_size_z_upper     = 0
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

# The time integration scheme...
MoL::ODE_Method = "RK3"

# The next section is required if you want some kind of confirmation
# on the console that your code is actually doing anything...
ActiveThorns = "CarpetIOBasic IOUtil CarpetReduce SymBase"
IO::out_fileinfo="none"
IOBasic::outInfo_every = 1
IOBasic::outInfo_vars = "{thorn_name}::phi {thorn_name}::psi"

# Get some data!
ActiveThorns = "CarpetIOASCII"
CarpetIOASCII::compact_format = false
IOASCII::out2D_every = 25
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "{thorn_name}::psi {thorn_name}::phi"
IOASCII::out2D_xz = "no"
IOASCII::out2D_yz = "no"
IOASCII::output_ghost_points = "no"

# Taking too big a time step can lead to unstable evolution.
# You can control that with dtfac and thorn Time.
ActiveThorns = "Time"
Time::dtfac = .2
"""
create_file("wave_toy.par",wave_toy_par_src,thorn_pars)

In [ ]:
!rm -fr ../simulations/wave_toy
!./simfactory/bin/sim create-run wave_toy.par --procs=1 --num-threads=1

In [ ]:
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# https://matplotlib.org/examples/color/colormaps_reference.html
cmap = cm.prism

In [ ]:
file_data = np.genfromtxt("../simulations/wave_toy/output-0000/wave_toy/psi.xy.asc")

In [ ]:
sets = np.unique(file_data[:,0])
width = 8
height = 4
print("sets=",sets)
mn, mx = np.min(file_data[:,12]),np.max(file_data[:,12])
#mn, mx = 0,1
for which in sets:
    g = file_data[file_data[:,0]==which,:]
    print("time=",which)
    x = g[:,5]
    y = g[:,6]
    z = g[:,12]
    zi = z.reshape(len(np.unique(y)),len(np.unique(x)))
    print('min/max=',np.min(zi),np.max(zi))
    plt.figure(figsize=(width, height))
    plt.imshow(zi[::-1,:],cmap,clim=(mn,mx))
    plt.show()

<h3>Question</h3>: Does the above simulation work properly if we use 2 procs? Why or why not? Try it.

In [ ]:
schedule_ccl_contents = """
## Schedule definitions for thorn {thorn_name}

## There won't be any storage allocated for a group
## unless a corresponding storage declaration exists
## for it in the schedule file. In square brackets,
## we specify the number of storage levels to allocate.
## These commented examples correspond to the commented
## examples in the interface file above.
# storage: force_group[3]
# storage: scalar_group

## Schedule a function defined in this thorn to run at the beginning
## of the simulation. The minimum you need to specify for a schedule
## item is what language it's written in. Choices are: C (which includes
## C++) and Fortran (which means Fortran90).
#SCHEDULE init_function at CCTK_INIT
#{
#  LANG: C
#}"Do some initial stuff"

storage: evol_group[3], rhs_group

SCHEDULE init_wave_toy in CCTK_INITIAL
{
  LANG: C
} "initialize data"

SCHEDULE evolve_wave_toy in MoL_CalcRHS
{
  LANG: C
  SYNC: {thorn_name}::evol_group
  SYNC: {thorn_name}::rhs_group
} "evolve data"

schedule wave_toy_registervars in MoL_Register
{
  LANG: C
}"Register funwave variables for MoL"

"""
thorn_dir = "arrangements/"+thorn_pars["arrangement_name"]+"/"+thorn_pars["thorn_name"]
create_file(thorn_dir+"/schedule.ccl",schedule_ccl_contents,thorn_pars)

In [ ]:
!time ./simfactory/bin/sim build --mdbkey make 'make -j2' --thornlist=./my_thorns.th | cat -

In [ ]:
!rm -fr ../simulations/wave_toy
!./simfactory/bin/sim create-run wave_toy.par --procs=2 --num-threads=1

In [ ]:
file_data = np.genfromtxt("../simulations/wave_toy/output-0000/wave_toy/psi.xy.asc")

In [ ]:
sets = np.unique(file_data[:,0])
width = 8
height = 4
print("sets=",sets)
mn, mx = np.min(file_data[:,12]),np.max(file_data[:,12])
#mn, mx = 0,1
for which in sets:
    g = file_data[file_data[:,0]==which,:]
    print("time=",which)
    x = g[:,5]
    y = g[:,6]
    z = g[:,12]
    zi = z.reshape(len(np.unique(y)),len(np.unique(x)))
    print('min/max=',np.min(zi),np.max(zi))
    plt.figure(figsize=(width, height))
    plt.imshow(zi[::-1,:],cmap,clim=(mn,mx))
    plt.show()

<h3>Exercises:</h3>
<ol>
<li>Add a parameter to param.ccl and use it to change the position of the Gaussian wave hump.</li>
<li>Advanced: Write a Conway's Game of Life thorn. For this thorn it does not make sense to use MoL. Schedule your routine in CCTK_EVOL.</li>
</ol>

<h2>Adaptive Mesh Refinement</h2>
Sometimes you care more about what is happening in one part of a grid than another. Wouldn't it be nice if you could
spend less time computing in regions that are less important? That way you could explore larger problems with less compute cycles.

Well, dream no more. Adaptive Mesh Refinement is supported in Cactus/Carpet. And, at least for the toy example we are
considering in this file, all you have to change to make use of it is your parameter file. (Actually, what we're going
to consider here is technically Fixed Mesh Refinement (FMR)).

<h3>How AMR works</h3>
For reasons of simplicity, we'll focus on a 2 refinement level system. Thus, we have a coarse (refinement level 0) and a fine level (refinement level 1). We start from a time t0, at which data on all refinement levels is consistent and valid.

* All data is initialized at time t = t0
* The coarse level is evolved forward to time to t = t0 + dt
* The fine level is evolved forward to time to t = t0 + dt/2
* Boundary data for the fine grid at time to t = t0+dt/2 is interpolated from the coarse grids at times t0 and t0+dt
* The fine level is evolved forward in time to t = t0 + dt
* Data from the fine level at t=t0+dt is copied back into the coarse grid at time t=t0+dt
* If not done, t0 += dt and repeat.

In [ ]:
wave_toy2_par_src = """
ActiveThorns = "{thorn_name} coordbase carpet cartgrid3d MoL"

Cactus::cctk_itlast = 250

# the output dir will be named after the parameter file name
IO::out_dir = $parfile

# The set of parameters required to specify the grid...
Carpet::domain_from_coordbase = "yes"
CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin =  0
CoordBase::xmax =  30
CoordBase::ymin =  0
CoordBase::ymax =  30
CoordBase::zmin =  0.0
CoordBase::zmax =  0.0
CoordBase::dx   =  0.25
CoordBase::dy   =  0.25

# Because we're doing a 2D code...
CoordBase::boundary_size_z_lower     = 0
CoordBase::boundary_size_z_upper     = 0
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

# The time integration scheme...
MoL::ODE_Method = "RK3"

# The next section is required if you want some kind of confirmation
# on the console that your code is actually doing anything...
ActiveThorns = "CarpetIOBasic IOUtil CarpetReduce SymBase"
IO::out_fileinfo="all"
IOBasic::outInfo_every = 1
IOBasic::outInfo_vars = "{thorn_name}::phi {thorn_name}::psi"

# Get some data!
ActiveThorns = "CarpetIOASCII"
CarpetIOASCII::compact_format = false
IOASCII::out2D_every = 25
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "{thorn_name}::psi {thorn_name}::phi"
IOASCII::out2D_xz = "no"
IOASCII::out2D_yz = "no"
IOASCII::output_ghost_points = "no"

# Taking too big a time step can lead to unstable evolution.
# You can control that with dtfac and thorn Time.
ActiveThorns = "Time"
Time::dtfac = .2

### MESH REFINEMENT ####
ActiveThorns = "CarpetRegrid2"
Carpet::init_fill_timelevels = "true"
Carpet::max_refinement_levels = 2
CarpetRegrid2::num_centres = 1
CarpetRegrid2::num_levels_1 = 2
CarpetRegrid2::position_x_1 = 8.0
CarpetRegrid2::position_y_1 = 8.0
CarpetRegrid2::position_z_1 = 0.0
CarpetRegrid2::radius_1[1] = 8.0

#MoL::disable_prolongation        = "no"

### Dissipation is numerical noise added to mitigate
### grid boundary effects seen in some AMR simulations.
#ActiveThorns = "Dissipation Spacemask SPHERICALSURFACE"
#Dissipation::vars = "{thorn_name}::psi {thorn_name}::phi"
#Dissipation::order = 1
"""
create_file("wave_toy2.par",wave_toy2_par_src,thorn_pars)

In [ ]:
!rm -fr ../simulations/wave_toy2
!./simfactory/bin/sim create-run wave_toy2.par --procs=2 --num-threads=1

In [ ]:
file_data = np.genfromtxt("../simulations/wave_toy2/output-0000/wave_toy2/psi.xy.asc")

In [ ]:
sets = np.unique(file_data[:,0])
width = 8
height = 4
# NOTE: I changed the colorscheme here
cmap = cm.prism
print("sets=",sets)

mn, mx = np.min(file_data[:,12]),np.max(file_data[:,12])
#mn, mx = 0,1
for which in sets:
    gg = file_data[file_data[:,0]==which,:]
    print("time=",which)
    for rl in range(10): # refinement levels
      g = gg[gg[:,2]==rl,:]
      x = g[:,5]
      y = g[:,6]
      z = g[:,12]
      if z.shape[0] == 0:
        continue
      print("refinement level =",rl)
      plt.figure(figsize=(width, height))
      zi = z.reshape(len(np.unique(y)),len(np.unique(x)))
      plt.imshow(zi[::-1,:],cmap,clim=(mn,mx))
      plt.show()

<h3>Excercise:</h3> Play around with the CarpetRegrid2 parameters. Attempt to add more levels of refinement and/or more centers of refinement.

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a> and <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1539567"> CCF 1539567</a></td><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>